## Travel Booking Platform – Product & Revenue Analytics

## Business Objective
The objective of this project is to analyze customer behavior, trip patterns,
pricing, discounts, and subscription adoption to identify revenue drivers,
retention opportunities, and product optimization strategies.

## Dataset Overview
- Dataset: trip_dataset.csv
- Rows: ~5,000 bookings

## Stakeholders
- Product Management
- Revenue & Pricing Team
- Growth & Marketing

## Key Decisions Supported
- Pricing & discount strategy
- Subscription targeting
- Premium package positioning

## Analysis Approach

The analysis follows a structured approach:
1. Data validation and exploratory analysis
2. Revenue and pricing cleaning
3. Feature engineering for behavioral segmentation
4. Business-driven analysis and insights


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('trip_dataset.csv')

# Basic validation
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])


df.head()

Rows: 5000
Columns: 15


,user_id,gender,age,age_group,city,destination,travel_type,previous_trips,is_subscriber,used_discount,base_price,discount_amount,final_amount,package_type,rating
0,25795,Male,45,36-45,Pune,Singapore,Leisure,3,1,1,20950,1833,19117,Standard,4.8
1,10860,Female,52,46-60,Pune,Manali,Adventure,3,0,0,16412,0,16412,Standard,4.7
2,86820,Male,28,26-35,Kolkata,Manali,Honeymoon,4,0,1,47294,3784,43510,Standard,4.1
3,64886,Female,48,46-60,Delhi,Jaipur,Business,3,0,1,35973,4522,31451,Standard,4.8
4,16265,Female,56,46-60,Kolkata,Bangkok,Adventure,4,0,1,4805,4095,710,Standard,3.2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          5000 non-null   int64  
 1   gender           5000 non-null   object 
 2   age              5000 non-null   int64  
 3   age_group        5000 non-null   object 
 4   city             5000 non-null   object 
 5   destination      5000 non-null   object 
 6   travel_type      5000 non-null   object 
 7   previous_trips   5000 non-null   int64  
 8   is_subscriber    5000 non-null   int64  
 9   used_discount    5000 non-null   int64  
 10  base_price       5000 non-null   int64  
 11  discount_amount  5000 non-null   int64  
 12  final_amount     5000 non-null   int64  
 13  package_type     5000 non-null   object 
 14  rating           5000 non-null   float64
dtypes: float64(1), int64(8), object(6)
memory usage: 586.1+ KB


In [5]:
df.describe(include='all')

,user_id,gender,age,age_group,city,destination,travel_type,previous_trips,is_subscriber,used_discount,base_price,discount_amount,final_amount,package_type,rating
count,5000.000000,5000,5000.000000,5000,5000,5000,5000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000,5000.000000
unique,NaN,2,NaN,4,6,8,4,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,Female,NaN,46-60,Pune,Bangkok,Business,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN
freq,NaN,2516,NaN,1599,857,671,1283,NaN,NaN,NaN,NaN,NaN,NaN,3284,NaN
mean,54561.890800,NaN,38.267000,NaN,NaN,NaN,NaN,3.015200,0.387600,0.594400,26569.32320,1556.159000,25013.164200,NaN,4.010800
std,26090.370121,NaN,12.004421,NaN,NaN,NaN,NaN,1.719989,0.487251,0.491057,13527.42035,1681.364451,13615.482055,NaN,0.584458
min,10055.000000,NaN,18.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,3001.00000,0.000000,-1631.000000,NaN,3.000000
25%,32309.500000,NaN,28.000000,NaN,NaN,NaN,NaN,2.000000,0.000000,0.000000,14943.25000,0.000000,13436.250000,NaN,3.500000
50%,54374.500000,NaN,38.000000,NaN,NaN,NaN,NaN,3.000000,0.000000,1.000000,26532.00000,946.000000,24961.000000,NaN,4.000000
75%,77180.500000,NaN,48.000000,NaN,NaN,NaN,NaN,4.000000,1.000000,1.000000,38135.25000,3027.250000,36631.250000,NaN,4.500000


### Initial Observations
- Dataset is well-structured with no missing values.
- Revenue and pricing variables show high variance, enabling segmentation.
- Ratings are mostly between 3 and 5, indicating generally positive user experience.

In [8]:
df.isnull().sum()

user_id            0
gender             0
age                0
age_group          0
city               0
destination        0
travel_type        0
previous_trips     0
is_subscriber      0
used_discount      0
base_price         0
discount_amount    0
final_amount       0
package_type       0
rating             0
dtype: int64

In [10]:
df['effective_revenue'] = df['final_amount'].clip(lower=0)

### Revenue Adjustment
Some bookings showed negative final amounts due to excessive discounts.
To avoid misleading revenue analysis, revenue was capped at zero.
This reflects real-world margin protection logic.

In [65]:
# Binary feature standardization
binary_cols = ['is_subscriber', 'used_discount']
df[binary_cols] = df[binary_cols].fillna(0).astype(int)

# Rating validation
df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]

# Revenue protection
df['effective_revenue'] = df['final_amount'].clip(lower=0)



 ## Filtered invalid ratings to maintain data integrity for better analysis.

## Feature Engineering for Behavioral Analysis

In [60]:
def trip_intensity(trips):
    if trips == 0:
        return 'First-Time'
    elif 1 <= trips <= 2:
        return 'Occasional'
    elif 3 <= trips <= 5:
        return 'Frequent'
    else:
        return 'Power User'

df['trip_intensity'] = df['previous_trips'].apply(trip_intensity)

## Trip intensity segments users by loyalty level, enabling targeted retention and subscription strategies.

In [ ]:
df['discount_dependency'] = np.where(
    df['base_price'] > 0,
    df['discount_amount'] / df['base_price'],
    0
)

In [22]:
df['premium_flag'] = (df['package_type'] == 'Premium').astype(int)

In [24]:
df['high_rating_flag'] = (df['rating'] >= 4.5).astype(int)

In [26]:
travel_revenue = (
    df.groupby('travel_type')['effective_revenue']
      .sum()
      .sort_values(ascending=False)
)

travel_revenue

travel_type
Honeymoon    32287058
Business     31818127
Leisure      31174603
Adventure    29802063
Name: effective_revenue, dtype: int64

## Business travel contributes the highest revenue, indicating strong corporate demand and repeat bookings.

 ## KPI 1: Subscriber vs Non-Subscriber

In [30]:
df.groupby('is_subscriber').agg(
    avg_revenue=('effective_revenue', 'mean'),
    total_revenue=('effective_revenue', 'sum'),
    avg_trips=('previous_trips', 'mean')
)

,avg_revenue,total_revenue,avg_trips
is_subscriber,,,
0,24935.507511,76352524,2.704115
1,25144.131579,48729327,3.506708


 ## KPI 2: Discount Impact

In [33]:
df.groupby('used_discount').agg(
    avg_revenue=('effective_revenue', 'mean'),
    avg_discount=('discount_amount', 'mean')
)

,avg_revenue,avg_discount
used_discount,,
0,26341.940335,0.000000
1,24111.842530,2618.033311


 ## KPI 3: Premium Package Performance

In [36]:
df.groupby('package_type').agg(
    avg_revenue=('effective_revenue', 'mean'),
    avg_rating=('rating', 'mean')
)

,avg_revenue,avg_rating
package_type,,
Premium,25027.836830,4.018065
Standard,25010.378502,4.007004


In [53]:
df.groupby(
    'trip_intensity',
    observed=True
)['premium_flag'].mean()

trip_intensity
First-Time    0.334802
Frequent      0.342285
Occasional    0.340689
Power User    0.364286
Name: premium_flag, dtype: float64

In [39]:
df.groupby('destination').agg(
    bookings=('user_id', 'count'),
    avg_rating=('rating', 'mean'),
    revenue=('effective_revenue', 'sum')
).sort_values(by='revenue', ascending=False)

,bookings,avg_rating,revenue
destination,,,
Bangkok,671,4.008197,16762979
Kerala,639,4.008607,16411094
Goa,649,3.978120,16041689
Jaipur,614,4.028502,15726887
Manali,640,3.993438,15719764
Singapore,616,4.045617,15096051
Paris,592,4.019088,14841800
Dubai,579,4.007772,14481587


## Key Business Recommendations

1. Focus subscription and premium campaigns on Frequent and Power Users, as they
   show higher repeat behavior and upgrade likelihood.
2. Reduce aggressive discounting on high-demand destinations to protect margins.
3. Promote Premium packages in Business and Leisure segments with higher ratings.
4. Use trip intensity and discount dependency to design personalized offers.

## Conclusion

This project demonstrates end-to-end product and revenue analytics,
covering data validation, cleaning, feature engineering, KPI analysis,
segmentation, and business recommendations.


In [69]:
!pip install psycopg2-binary sqlalchemy

In [77]:
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
# Replace placeholders with your actual details
username = "postgres"      # default user
password = "1234" # the password you set during installation
host = "localhost"         # if running locally
port = "5433"              # default PostgreSQL port
database = "Tourism"    # the database you created in pgAdmin

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# Step 2: Load DataFrame into PostgreSQL
table_name = "customer"   # choose any table name
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'customer' in database 'Tourism'.
